# Квантизация модели под CPU

# Скачивание и импорт библиотек

In [ ]:
!pip install optimum[onnxruntime]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia

In [ ]:
# @title Импорт всех библиотек
from huggingface_hub import login, ModelCard, ModelCardData, create_repo, whoami, HfApi

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.onnxruntime import (
    AutoQuantizationConfig,
    ORTModelForSequenceClassification,
    ORTQuantizer
)
from optimum.pipelines import pipeline
from torch import device
import time

In [ ]:
# @title Авторизация
login("---------------")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Преобразование модели и её вывод

In [ ]:
# Имя модели
model_id = "khvatov/ru_toxicity_detector"
save_dir = "final_dir"

# Экспортирую модель в ONNX
model = ORTModelForSequenceClassification.from_pretrained(model_id, export=True, from_transformers=True)
# Необходимо использовать CPU
model.to(device('cpu'))
# Загружаю токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Сохраняю модель
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


# -----Это на потом, если что
# Config квантования
# qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=True)
# quantizer = ORTQuantizer.from_pretrained(model)

# # Сохраняю модель с квантизацией
# quantizer.quantize(save_dir=save_dir, quantization_config=qconfig)

The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

Framework not specified. Using pt to export the model.


model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: BertForSequenceClassification *****
Using framework PyTorch: 2.3.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False


('final_dir/tokenizer_config.json',
 'final_dir/special_tokens_map.json',
 'final_dir/vocab.txt',
 'final_dir/added_tokens.json',
 'final_dir/tokenizer.json')

In [ ]:
# @title Проверка
start_time = time.time()

# model = ORTModelForSequenceClassification.from_pretrained("Oxtiz/final_try")

onnx_clx = pipeline("text-classification", model=model, tokenizer=tokenizer, accelerator="ort")
text = "Ты говно. Ты мне нравишься"
pred = onnx_clx(text)
print(pred)
print("--- %s seconds ---" % (time.time() - start_time))

[{'label': 'LABEL_1', 'score': 0.997707724571228}]
--- 0.008086204528808594 seconds ---


# Загрузка на HuggingFace

In [ ]:
# @title Создаю карточку
hub_name = "onnx_ru_toxicity_detector"

card_data = ModelCardData(language='ru')
card = ModelCard.from_template(
    card_data,
    model_id=hub_name,
    developers="Я",
)

user = whoami()['name']
repo_id = f'{user}/{hub_name}'
url = create_repo(repo_id, exist_ok=True)

card.push_to_hub(repo_id)

CommitInfo(commit_url='https://huggingface.co/Oxtiz/onnx_ru_toxicity_detector/commit/e244dda152a3db0a59349008785f3d0fcfd1f9a4', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='e244dda152a3db0a59349008785f3d0fcfd1f9a4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# @title Попытка загрузить директорию в репу

# Инициализация APi
api = HfApi()

# Загрузка папки
api.upload_folder(
    folder_path=save_dir,
    repo_id=repo_id
)

model.onnx:   0%|          | 0.00/117M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Oxtiz/onnx_ru_toxicity_detector/commit/0dbff7ab443f0742118f213ded01bae57ca6cb28', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0dbff7ab443f0742118f213ded01bae57ca6cb28', pr_url=None, pr_revision=None, pr_num=None)